# Walk-through FitLins Analysis

This document demonstrates the command line to run a FitLins analysis.

We use the [OpenNeuro](https://openneuro.org) dataset [ds000003](https://openneuro.org/datasets/ds000003/),
with [fMRIPrep](https://fmriprep.rtfd.io)-preprocessed results at
[///labs/poldrack/ds003_fmriprep](http://datasets.datalad.org/?dir=/labs/poldrack/ds003_fmriprep).

## Preamble

We will use [Datalad](https://www.datalad.org) to manage our data, which in turn depends on `git-annex`. To install `git-annex`, uncomment and run one of the following:

In [1]:
# Debian
# apt-get install git-annex

In [2]:
# Linux with Conda
# conda install -y git-annex

In [3]:
# OSX
# open https://git-annex.branchable.com/install/OSX/

Datalad and FitLins are Python tools.
To ensure that everything installs correctly, we'll upgrade the [pip](https://pip.pypa.io/en/stable/) package manager
and the [setuptools](https://setuptools.readthedocs.io/en/latest/) utilities.

We will also install unreleased versions of Datalad and [Nistats](https://nistats.github.io/) to make things run a little more smoothly.

In [4]:
pip install -q --upgrade pip setuptools
pip install -q --upgrade "datalad @ git+https://github.com/datalad/datalad@5f5342ce2e63b91d54f75edb87d97129a916d9ef" \
                         "nistats @ git+https://github.com/nistats/nistats.git@b69e127020df05aa8ba540f9a8d799e1dad3d602"

## Install data and FitLins

First, install the [dataset](http://datasets.datalad.org/?dir=/labs/poldrack/ds003_fmriprep) we'll be working with.

In [5]:
# Set environment variable to make outputs more friendly
export DATALAD_UI_PROGRESSBAR=log
export NO_COLOR="1"

datalad install -r ///labs/poldrack/ds003_fmriprep
datalad update ds003_fmriprep

[INFO   ] Cloning http://datasets.datalad.org/labs/poldrack/ds003_fmriprep [1 other candidates] into '/Users/markiewicz/Projects/crn/fitlins/examples/notebooks/ds003_fmriprep' 
[INFO   ]  Cloning (counting objects) 100 objects done in 9.799e-05 sec at 1020512 objects/sec 
[INFO   ]  Cloning (compressing objects) 23339 objects done in 0.182001 sec at 128236 objects/sec 
[INFO   ]  Cloning (receiving objects) 44574 objects done in 0.569176 sec at 78313 objects/sec 
[INFO   ]  Cloning (resolving stuff) 10729 objects done in 0.438001 sec at 24495 objects/sec 
install(ok): /Users/markiewicz/Projects/crn/fitlins/examples/notebooks/ds003_fmriprep (dataset)
[INFO   ] Installing <Dataset path=/Users/markiewicz/Projects/crn/fitlins/examples/notebooks/ds003_fmriprep> recursively 
[INFO   ] Cloning http://datasets.datalad.org/labs/poldrack/ds003_fmriprep/sourcedata/.git [2 other candidates] into '/Users/markiewicz/Projects/crn/fitlins/examples/notebooks/ds003_fmriprep/sourcedata' 
[INFO   ]  Cloni

Fetch the preprocessed BOLD series and calculated confounds for the all subjects.

In [6]:
datalad get ds003_fmriprep/sub-*/func/*_space-MNI152NLin2009cAsym_desc-*.nii.gz \
            ds003_fmriprep/sub-*/func/*_desc-confounds_*.tsv

[INFO   ]  sub-01/func .. bold.nii.gz 211.2 MB done in 17 seconds at 11.8 MB/sec 
[INFO   ]  sub-02/func .. bold.nii.gz 212.0 MB done in 24 seconds at 8.6 MB/sec 
[INFO   ]  sub-03/func .. bold.nii.gz 222.7 MB done in 24 seconds at 9.0 MB/sec 
[INFO   ]  sub-04/func .. bold.nii.gz 224.0 MB done in 30 seconds at 7.4 MB/sec 
[INFO   ]  sub-05/func .. bold.nii.gz 240.6 MB done in 20 seconds at 11.9 MB/sec 
[INFO   ]  sub-06/func .. bold.nii.gz 214.9 MB done in 23 seconds at 9.3 MB/sec 
[INFO   ]  sub-08/func .. bold.nii.gz 238.4 MB done in 25 seconds at 9.4 MB/sec 
[INFO   ]  sub-07/func .. bold.nii.gz 242.6 MB done in 31 seconds at 7.7 MB/sec 
[INFO   ]  sub-09/func .. bold.nii.gz 223.6 MB done in 20 seconds at 10.7 MB/sec 
[INFO   ]  sub-10/func .. bold.nii.gz 235.5 MB done in 28 seconds at 8.1 MB/sec 
[INFO   ]  sub-12/func .. bold.nii.gz 203.5 MB done in 20 seconds at 9.9 MB/sec 
[INFO   ]  sub-11/func .. bold.nii.gz 217.6 MB done in 24 seconds at 8.7 MB/sec 
[INFO   ]  sub-13/func ..

Finally, install FitLins via pip. See the [Installation](https://fitlins.readthedocs.io/en/latest/installation.html)
section of the documentation for instructions on using Docker or Singularity. For the purposes of this tutorial,
however, we'll assume you can install to your current Python environment without issue.

In [7]:
pip install -q --upgrade fitlins==0.4.0

## Dataset structure

This dataset is organized as an fMRIPrep derivatives directory, with the original dataset included as the `sourcedata/` sub-dataset.
The following command shows just the directories for the first three subjects. (Will probably not work with earlier versions than `tree 1.8.0`.)

In [8]:
tree -P "sub-0[123]*.nii.gz" -I "figures|log" --prune --noreport ds003_fmriprep | grep -v -- '->'

ds003_fmriprep
├── sourcedata
│   ├── sub-01
│   │   ├── anat
│   │   └── func
│   ├── sub-02
│   │   ├── anat
│   │   └── func
│   └── sub-03
│       ├── anat
│       └── func
├── sub-01
│   ├── anat
│   └── func
├── sub-02
│   ├── anat
│   └── func
└── sub-03
    ├── anat
    └── func


### Model

We will estimate the model found in `../models/ds000003/models/model-001_smdl.json`:

In [1]:
cat ../models/ds000003/models/model-001_smdl.json

{
  "Name": "ds003_model001",
  "Description": "",
  "Input": {
    "task": "rhymejudgment"
  },
  "Steps": [
    {
      "Level": "run",
      "Transformations": [
        {
          "Name": "Factor",
          "Input": [
            "trial_type"
          ]
        },
        {
          "Name": "Convolve",
          "Input": ["trial_type.word", "trial_type.pseudoword"],
          "Model": "spm"
        }
      ],
      "Model": {
        "X": [
          "trial_type.word",
          "trial_type.pseudoword",
          "framewise_displacement",
          "trans_x", "trans_y", "trans_z", "rot_x", "rot_y", "rot_z",
          "a_comp_cor_00", "a_comp_cor_01", "a_comp_cor_02",
          "a_comp_cor_03", "a_comp_cor_04", "a_comp_cor_05"
        ]
      },
      "DummyContrasts": {
        "Conditions": ["trial_type.word", "trial_type.pseudoword"],
        "Type": "t"
        },
      "Contrasts": [
        {
          "Name": "word_gt_pseudo",
          "ConditionList": [
            "tri

## Running FitLins

A BIDS App takes a BIDS dataset, a target directory and an analysis level. Use the BIDS-Raw dataset (`ds003_fmriprep/sourcedata`) as the dataset, and we'll create a new directory `output`. The `dataset` level indicates the highest level of analysis, so `run`, `session` and `subject` will be run, if found in the model.

In addition, we pass `ds003_fmriprep` as with the `--derivatives` (or `-d`) flag, indicating that preprocessed data should be found there. We will also pass the model from `openfmri/ds000003/model.json`

In [10]:
fitlins ds003_fmriprep/sourcedata output/ dataset \
    --derivatives $PWD/ds003_fmriprep \
    --model $PWD/../models/ds000003/models/model-001_smdl.json \
    --smoothing 5:run \
    -w scratch \
    --n-cpus 2

Failed to import duecredit due to No module named 'duecredit'
190524-12:49:11,669 nipype.workflow INFO:
	 Workflow fitlins_wf settings: ['check', 'execution', 'logging', 'monitoring']
190524-12:49:11,702 nipype.workflow INFO:
	 Running in parallel.
190524-12:49:11,708 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
190524-12:49:11,813 nipype.workflow INFO:
	 [Node] Setting-up "fitlins_wf.loader" in "/Users/markiewicz/Projects/crn/fitlins/examples/notebooks/scratch/fitlins_wf/loader".
190524-12:49:11,821 nipype.workflow INFO:
	 [Node] Running "loader" ("fitlins.interfaces.bids.LoadBIDSModel")
190524-12:49:13,710 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 14.20/14.40, Free processors: 1/2.
                     Currently running:
                       * fitlins_wf.loader
Captured warning (<class 'pandas.core.common.SettingWithCopyWarning'>): 
A value is trying to be s